🟦 CELL 1 – IMPORT + PATH CONFIG (Letakkan di awal file)

In [7]:
import os
import pandas as pd
import numpy as np

# Set folder input/output
FOLDER_CLUSTERED = r"E:\1.Clustering_TA\3output_clustering\01_clustered_point_cloud"
FOLDER_TORSO = r"E:\1.Clustering_TA\3output_clustering\02_torso_per_frame"
FOLDER_GAITFRAME = r"E:\1.Clustering_TA\dataset\gaitframe"

# Pastikan folder output ada
os.makedirs(FOLDER_GAITFRAME, exist_ok=True)


🟦 CELL 2 – Helper untuk konversi timestamp ke detik

In [8]:
def convert_timestamp_to_seconds(ts):
    """
    Konversi string timestamp 'HH:MM:SS.microsec' ke detik float.
    """
    try:
        h, m, s = ts.split(":")
        return int(h) * 3600 + int(m) * 60 + float(s)
    except:
        return np.nan


🟦 CELL 3 – Fungsi utama build_gait_frame()

In [9]:
def build_gait_frame(df_clustered, df_torso):
    """
    Membangun gait frame ala Fig. 6 jurnal in-home:
    - menggabungkan torso per frame
    - menghitung doppler torso dari point cloud torso cluster
    - mengambil timestamp per frame dari df_clustered
    """

    # Precompute mapping frame -> timestamp (ambil yang pertama per frame)
    frame_to_ts = (
        df_clustered
        .sort_values(["frame", "timestamp"])
        .groupby("frame")["timestamp"]
        .first()
        .to_dict()
    )

    rows = []

    for idx, row in df_torso.iterrows():
        frame = int(row["frame"])
        torso_id = int(row["torso_cluster_id"])

        # skip frame gap → doppler_torso = NaN
        if torso_id == -1:
            doppler_torso = np.nan
        else:
            pts = df_clustered[
                (df_clustered["frame"] == frame) &
                (df_clustered["cluster_id"] == torso_id)
            ]
            if len(pts) > 0:
                doppler_torso = pts["doppler"].median()
            else:
                doppler_torso = np.nan

        # timestamp diambil dari clustered (kalau ada)
        ts_raw = frame_to_ts.get(frame, None)
        ts_sec = convert_timestamp_to_seconds(ts_raw) if ts_raw is not None else np.nan

        rows.append({
            "frame": frame,
            "timestamp": ts_raw,
            "timestamp_sec": ts_sec,
            "torso_x": row["torso_x"],
            "torso_y": row["torso_y"],
            "torso_z": row["torso_z"],
            "doppler_torso": doppler_torso,
            "is_gap": 1 if torso_id == -1 else 0,
        })

    df_gait = pd.DataFrame(rows)
    return df_gait



🟦 CELL 4 – Fungsi pemrosesan otomatis untuk semua trial

In [10]:
def process_all_gait_frames(
    folder_cluster=FOLDER_CLUSTERED, 
    folder_torso=FOLDER_TORSO, 
    folder_out=FOLDER_GAITFRAME
):
    files = sorted(os.listdir(folder_torso))
    results = []

    for fname in files:
        if not fname.endswith("_torso.csv"):
            continue

        torso_path = os.path.join(folder_torso, fname)
        base = fname.replace("_clustered_torso.csv", "")

        cluster_file = base + "_clustered.csv"
        cluster_path = os.path.join(folder_cluster, cluster_file)

        if not os.path.exists(cluster_path):
            print(f"SKIP (cluster file not found): {cluster_path}")
            continue

        print(f"\n=== PROSES GAIT FRAME: {base} ===")

        df_torso = pd.read_csv(torso_path)
        df_cluster = pd.read_csv(cluster_path)

        df_gait = build_gait_frame(df_cluster, df_torso)

        out_csv = os.path.join(folder_out, f"{base}_gait_frame.csv")
        df_gait.to_csv(out_csv, index=False)

        print(f"Saved gait frame → {out_csv}")

        results.append({
            "trial": base,
            "frames": len(df_gait),
            "gap_frames": df_gait["is_gap"].sum()
        })

    return pd.DataFrame(results)


🟦 CELL 5 – Jalankan pipeline otomatis

In [11]:
df_summary = process_all_gait_frames()
df_summary



=== PROSES GAIT FRAME: Afi_Jalan10 ===
Saved gait frame → E:\1.Clustering_TA\dataset\gaitframe\Afi_Jalan10_gait_frame.csv

=== PROSES GAIT FRAME: Afi_Jalan11 ===
Saved gait frame → E:\1.Clustering_TA\dataset\gaitframe\Afi_Jalan11_gait_frame.csv

=== PROSES GAIT FRAME: Afi_Jalan12 ===
Saved gait frame → E:\1.Clustering_TA\dataset\gaitframe\Afi_Jalan12_gait_frame.csv

=== PROSES GAIT FRAME: Afi_Jalan13 ===
Saved gait frame → E:\1.Clustering_TA\dataset\gaitframe\Afi_Jalan13_gait_frame.csv

=== PROSES GAIT FRAME: Afi_Jalan14 ===
Saved gait frame → E:\1.Clustering_TA\dataset\gaitframe\Afi_Jalan14_gait_frame.csv

=== PROSES GAIT FRAME: Afi_Jalan15 ===
Saved gait frame → E:\1.Clustering_TA\dataset\gaitframe\Afi_Jalan15_gait_frame.csv

=== PROSES GAIT FRAME: Afi_Jalan16 ===
Saved gait frame → E:\1.Clustering_TA\dataset\gaitframe\Afi_Jalan16_gait_frame.csv

=== PROSES GAIT FRAME: Afi_Jalan17 ===
Saved gait frame → E:\1.Clustering_TA\dataset\gaitframe\Afi_Jalan17_gait_frame.csv

=== PROSES GAIT

,trial,frames,gap_frames
0,Afi_Jalan10,79,2
1,Afi_Jalan11,82,6
2,Afi_Jalan12,110,16
3,Afi_Jalan13,59,0
4,Afi_Jalan14,78,11
...,...,...,...
211,Miftah_Jalan71,265,12
212,Miftah_Jalan72,366,9
213,Miftah_Jalan7,102,0
214,Miftah_Jalan8,101,4
